# **Lorem Ipsum Dolor Sit Amet**

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec vestibulum tortor tortor, sed lobortis velit pharetra et. Mauris at lectus quis lorem viverra cursus. Praesent a efficitur lectus, vitae aliquet urna. Phasellus aliquam felis non consequat eleifend. Nulla ut feugiat ex. Suspendisse non justo vel neque dignissim vestibulum. Praesent feugiat vel nisi eu ullamcorper. Sed tincidunt mollis ex. 

**Authors:**
- [Adrian Flores](https://github.com/adrianRFlores)
- [Andrea Ramirez](https://github.com/Andrea-gt)

***

## **(1) Import Libraries** ⬇️

In [1]:
#!pip install unidecode

In [2]:
# Data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from unidecode import unidecode

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

# ===== ===== Reproducibility Seed ===== =====
# Set a fixed seed for the random number generator for reproducibility
random_state = 42

# Set matplotlib inline
%matplotlib inline

# Set default figure size
plt.rcParams['figure.figsize'] = (6, 4)

# Define custom color palette
palette = sns.color_palette("viridis", 12)

# Set the style of seaborn
sns.set(style="whitegrid")

## **(2) Data Upload** 📄